<a href="https://colab.research.google.com/github/nadkomash/malaria_recognition/blob/main/malara_recognition_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from skimage.transform import resize



Loading data from tfds

In [ ]:
malaria, info = tfds.load('malaria',split='train[:2500]',with_info=True)
fig = tfds.show_examples(malaria,info)


Logistic Regression on data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

converting to numpy array , resizing and flattenining.

In [ ]:
images = []
labels =[]
for example in tfds.as_numpy(malaria):
  images.append(resize(example['image'],(100,100)))
  labels.append(example['label'])
  if len(images)==2500:
    break

X = np.array(images)
y = np.array(labels)

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.20,random_state=42)

X_train = X_train.flatten().reshape((X_train.shape[0],-1))
X_test = X_test.flatten().reshape((X_test.shape[0],-1))

print(X_train.shape)
print(X_test.shape)



In [ ]:
lr = LogisticRegression(max_iter=1000,random_state=42)
lr = lr.fit(X_train,y_train)
test_acc = lr.score(X_test,y_test)
print(f'Test Accuracy for Logistic Regression : {test_acc:.3f}')


modeling with keras NN

In [ ]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense , Dropout

In [ ]:
model = Sequential()
model.add(Dense(300 ,input_shape=(30000,), activation= 'relu',name='my_dense_layer'))
model.add(Dense(100 ,activation= 'relu'))
model.add(Dense(50 ,activation= 'relu'))
model.add(Dense(1 ,activation= 'sigmoid'))
model.summary()



In [ ]:
model.layers[0].name
W1,b1 = model.get_layer('my_dense_layer').get_weights()
print(W1.shape)
W1

Compiling

In [ ]:
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=["accuracy"])

Fitting

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [EarlyStopping(monitor = 'val_loss',patience=5,restore_best_weights=True)]

history1 = model.fit(X_train,y_train, batch_size= 100, epochs=50 ,validation_split=0.1, callbacks=callbacks,verbose=0)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.DataFrame(history1.history).plot(figsize=(10,6))
plt.grid(True)
plt.show()

Evaluation

In [ ]:
test_loss , test_acc = model.evaluate(X_test,y_test,verbose=False)
print(f'Test accuracy for NN: {test_acc:.3f}')

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = (model.predict(X_test , verbose=0)>0.5).astype(int).reshape(y_test.shape)
c_m  = pd.DataFrame(
    confusion_matrix(y_test,y_pred),
    index=['true:yes','true:no'],
    columns=['pred:yes','pred:no'])
print(c_m)


Tuning parameters: (Keras tuner)

In [ ]:
!pip install scikeras

In [ ]:
!pip install keras

In [ ]:
from tensorflow.keras.layers import  InputLayer
from tensorflow.keras.optimizers import SGD
from scikeras.wrappers import KerasClassifier
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:

# Define malaria model builder function
def malaria_model(n_hidden, n_neurons, learning_rate):
    model = Sequential()

    # Input layer: here input_shape=(30000,) means your data has 30,000 features per sample
    model.add(InputLayer(input_shape=(30000,)))

    # Hidden layers
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation='relu'))

    # Output layer (binary classification)
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(
        loss="binary_crossentropy",
        optimizer=SGD(learning_rate=learning_rate),
        metrics=["accuracy"]
    )


    return model

# Wrap the Keras model for scikit-learn compatibility
keras_clf = KerasClassifier(
    model=malaria_model,
    n_hidden=1,
    n_neurons=30,
    learning_rate=3e-3,
    epochs=50
)


params = {
    'n_hidden': [0, 1, 2, 3],
    'n_neurons': np.arange(1, 100),
    'learning_rate': reciprocal(3e-4, 3e-2)
}


callbacks = [EarlyStopping(monitor = 'val_loss',patience=5,restore_best_weights=True)]

history1 = model.fit(X_train,y_train, batch_size= 100, epochs=50 ,validation_split=0.1, callbacks=callbacks,verbose=0)
# Randomized search with cross-validation
rnd_search_cv = RandomizedSearchCV(estimator=keras_clf, param_distributions=params, n_iter=10, cv=5)
# Fit the search
rnd_search_cv.fit( X_train, y_train, validation_split=0.1,callbacks=callbacks)

# Print results
print(f"Best test accuracy: {rnd_search_cv.best_score_:.2f}")
print(f"Best params: {rnd_search_cv.best_params_}")

